In [153]:
import os
os.chdir('/Users/peytonhall/Documents/qbio_490_pbhall/analysis_data')
import cptac
import numpy as np

In [154]:
cptac.download(dataset="Ccrcc")
ccrcc = cptac.Ccrcc()
ccrcc.list_data()

Below are the dataframes contained in this dataset and their dimensions:

clinical
	194 rows
	171 columns
CNV
	110 rows
	19285 columns
followup
	352 rows
	27 columns
medical_history
	370 rows
	4 columns
methylation
	107 rows
	15885 columns
phosphoproteomics
	194 rows
	81550 columns
phosphoproteomics_gene
	194 rows
	6127 columns
proteomics
	194 rows
	11710 columns
somatic_mutation
	8350 rows
	3 columns
transcriptomics
	185 rows
	19275 columns


In [167]:
#feature selection 
clinical_data = ccrcc.get_clinical()
#clinical_data

protein_data = ccrcc.get_proteomics()
protein_data.columns= protein_data.columns.get_level_values(0)


rna_data = ccrcc.get_transcriptomics()

#rna_data

In [168]:
#clean clinical data
import pandas as pd
NaN_mask = clinical_data.loc[:,'tumor_stage_pathological'].isna()


clinical_mask = clinical_data.loc[~NaN_mask, :]
protein_mask = protein_data.loc[~NaN_mask, :]
rna_mask = rna_data.loc[~NaN_mask, :]

protein_mask = protein_mask.dropna(axis=1)
rna_mask = rna_mask.replace(0, np.nan)
rna_mask =rna_mask.dropna(axis=1)

In [169]:
#feature selection 
stage_df = clinical_mask[["tumor_stage_pathological"]].copy
stage_df

<bound method NDFrame.copy of Name       tumor_stage_pathological
Patient_ID                         
C3L-00004                 Stage III
C3L-00010                   Stage I
C3L-00011                  Stage IV
C3L-00026                   Stage I
C3L-00079                 Stage III
...                             ...
C3N-01646                 Stage III
C3N-01648                  Stage II
C3N-01649                 Stage III
C3N-01651                  Stage II
C3N-01808                   Stage I

[110 rows x 1 columns]>

In [170]:
#feature selection 
stage_I = clinical_mask.loc[:, 'tumor_stage_pathological'] == 'Stage I'
stage_I

stage_III = clinical_mask.loc[:, 'tumor_stage_pathological'] == 'Stage III'
stage_III

rna_mask = np.log2(rna_mask)
print(rna_mask)

Name            A1BG      A1CF       A2M     A2ML1    A4GALT     A4GNT  \
Patient_ID                                                               
C3L-00004  -0.006744  4.059860  8.464600 -4.422478  4.104075 -0.158254   
C3L-00010  -0.557668  4.060282  8.488155 -3.692446  3.761339  0.802391   
C3L-00011  -1.495945 -2.025585  7.794905 -4.041315  0.402175 -2.536424   
C3L-00026   1.346971  4.031001  7.834675 -3.544831  2.976069  0.534969   
C3L-00079   2.122741  2.280647  8.103761 -3.232986  2.778840  1.225325   
...              ...       ...       ...       ...       ...       ...   
C3N-01646   1.497392  1.443675  8.220375 -3.366738  3.302403 -1.293858   
C3N-01648   2.809102 -5.774615  9.454755 -4.092753  2.150166 -1.231377   
C3N-01649   1.145973  1.162063  9.264360 -4.332497  3.309683  0.194761   
C3N-01651   0.342610  3.734611  8.400793 -2.375570  3.976759 -1.010459   
C3N-01808  -0.146368  3.599077  9.040864 -4.160852  3.912500  1.508759   

Name            AAAS      AACS     AA

In [175]:
diff_rna1 = rna_mask.loc[stage_I, :].mean() - rna_mask.loc[stage_III, :].mean()
#HERE ISSUE IS HERE
#rna1_shared_mask = diff_rna1.index.isin(protein_mask.columns)
top_51_rna = diff_rna1.sort_values(ascending=False)[:5].index
#stage 1 'DAO', 'IYD', 'NOS1', 'HGFAC', 'RTL1'

diff_rna3 = rna_mask.loc[stage_III, :].mean() - rna_mask.loc[stage_I, :].mean()
#rna3_shared_mask = diff_rna3.index.isin(protein_mask.columns)
top_53_rna = np.abs(diff_rna3).sort_values(ascending=False)[:5].index
#stage 3 'JSRP1', 'SFRP2', 'SEPT3', 'INHBE', 'TRIM74'
print(top_51_rna, top_53_rna)

#get top 5 differentially expressed proteins
#diff_proteins = protein_mask.loc[stage_I, :].mean()- protein_mask.loc[stage_III, :].mean()
#top5_proteins = diff_proteins.sort_values(ascending= False)[:5].index

Index(['AJAP1', 'DPEP1', 'GALNT5', 'ATRNL1', 'IGF2'], dtype='object', name='Name') Index(['AJAP1', 'DPEP1', 'GALNT5', 'IL20RB', 'RYR2'], dtype='object', name='Name')


In [176]:
x_data = rna_mask[['AJAP1', 'DPEP1', 'GALNT5', 'IL20RB', 'RYR2']]
x_data

Name,AJAP1,DPEP1,GALNT5,IL20RB,RYR2
Patient_ID,,,,,
C3L-00004,-2.519207,-1.239811,-5.281123,2.685910,-0.721420
C3L-00010,-0.069282,4.515644,-1.685391,0.394540,-2.134346
C3L-00011,0.411676,-1.955016,-2.430473,5.329663,3.102518
C3L-00026,0.177869,4.635430,2.729468,1.139609,-1.565029
C3L-00079,-1.161677,0.376214,-3.413559,0.441469,-0.421905
...,...,...,...,...,...
C3N-01646,-1.251963,-0.946572,-0.635367,7.399686,-0.645290
C3N-01648,-2.212814,2.734819,1.865296,0.789974,-1.221495
C3N-01649,-0.097150,3.354027,-1.442680,-0.220637,2.057367


In [177]:
y_data = clinical_mask.loc[:,'tumor_stage_pathological']
y_data

Patient_ID
C3L-00004    Stage III
C3L-00010      Stage I
C3L-00011     Stage IV
C3L-00026      Stage I
C3L-00079    Stage III
               ...    
C3N-01646    Stage III
C3N-01648     Stage II
C3N-01649    Stage III
C3N-01651     Stage II
C3N-01808      Stage I
Name: tumor_stage_pathological, Length: 110, dtype: object

In [182]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
#scaling 
diff_prot1 = protein_mask.loc[stage_I, :].mean() - protein_mask.loc[stage_III, :].mean()
top_51_prot = diff_prot1.sort_values(ascending=False)[:5].index
#stage 1 'FTL', 'HBZ', 'HBA2', 'CMA1', 'HBB'

diff_prot3 = protein_mask.loc[stage_III, :].mean() - protein_mask.loc[stage_I, :].mean()
top_53_prot = np.abs(diff_prot3).sort_values(ascending=False)[:5].index
#stage 3 
print(top_51_prot, top_53_prot)

z_data = protein_mask[['FTL', 'HBZ', 'HBA2', 'CMA1', 'HBB']]
z_data

cat_x_z = pd.concat([x_data, z_data], axis=1)
scaler = StandardScaler()
cat_x_z = scaler.fit_transform(cat_x_z)

encoder = LabelEncoder()
y_data = encoder.fit_transform(y_data)

#np.log2(x_data)

Index(['FTL', 'HBZ', 'HBA2', 'CMA1', 'HBB'], dtype='object', name='Name') Index(['FTL', 'HBZ', 'HBA2', 'CMA1', 'HBB'], dtype='object', name='Name')


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
#KNeighborsClassifier 
classifier = KNeighborsClassifier()
X_train, X_test, y_train, y_test = train_test_split(cat_x_z, y_data, train_size=0.75)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test.to_numpy()) 
